# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Pizza restaurant** in **İstanbul**, Turkey.

Since there are lots of restaurants in Istanbul we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Pizza restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Pizza restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* educational status of people living in the district. I got the education status information from https://biruni.tuik.gov.tr/medas/. I think people with a high level of education will eat at more restaurants or order takeaways.


***Required Library***

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim 
import folium

ModuleNotFoundError: No module named 'geopy'

***Coordinate and education level information of the districts of Istanbul***

You can find the information here as csv in the repository.

In [ ]:
df_egitim=pd.read_csv("SecimSonuclari_EgitimDurumu.csv")
df_egitim_map=pd.read_csv("EğitimDurumuMapping.csv")
df_longlat=pd.read_csv("Ilce_Lat_long.csv")
df_egitim=pd.melt(df_egitim,id_vars=["İlçe"],value_vars=df_egitim.columns[1:]).rename(columns={"variable":"Sınıf","value":"Kişi"})
df_egitim=pd.merge(df_egitim,df_egitim_map,on="Sınıf")
df_egitim=df_egitim.groupby(["İlçe","Education Status 2"], as_index=False)["Kişi"].sum()
df_egitim=df_egitim.pivot(index="İlçe",columns="Education Status 2",values="Kişi").reset_index()
df_egitim.rename_axis(None, axis = 1,inplace=True)
#Eğitimli ve eğitimsiz kitlelerin ilçelere göre oranını bulacağım.
df_egitim["Trained%"]=df_egitim["Trained"]/(df_egitim["Trained"]+df_egitim["Un-Trained"])
df_egitim["Un-Trained%"]=df_egitim["Un-Trained"]/(df_egitim["Trained"]+df_egitim["Un-Trained"])
df_egitim=pd.merge(df_egitim,df_longlat,on="İlçe")
df_egitim=df_egitim.rename(columns={"İlçe":"Districts"})
df_egitim.head()

In [ ]:
# Geograpical coordinate of Istanbul
address = 'Taksim Square,Istanbul'

geolocator = Nominatim(user_agent="Istanbul")
location = geolocator.geocode(address)
latitude_istanbul = location.latitude
longitude_istanbul = location.longitude
print('The geograpical coordinate of Istanbul are {}, {}.'.format(latitude_istanbul, longitude_istanbul))

In [ ]:
map_istanbul = folium.Map(location=[latitude_istanbul,longitude_istanbul],zoom_start=10)
label = address
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
[latitude_istanbul,longitude_istanbul],
radius=5,
popup=label,
color='blue',
fill=True,
fill_color='#3186cc',
fill_opacity=0.7,
parse_html=False).add_to(map_istanbul)
map_istanbul

In [ ]:
def replace_eng_chr(text): # Replace Turkish Char To English
    text=text.replace("ğ","g")
    text=text.replace("Ğ","G")
    text=text.replace("ı","i")
    text=text.replace("İ","I")
    text=text.replace("ö","o")
    text=text.replace("Ö","O")
    text=text.replace("ü","u")
    text=text.replace("Ü","U")
    text=text.replace("ş","s")
    text=text.replace("Ş","S")
    text=text.replace("ç","c")
    text=text.replace("Ç","C")
    return text

In [ ]:
#Map of Istanbul districts
for lat,lng,ilce in zip(df_egitim['Latitude'],df_egitim['Longitude'],df_egitim['Districts']):
    label = replace_eng_chr(ilce)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [float(lat),float(lng)],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_istanbul)
map_istanbul